<h1>Example post quantum secure communication between ALice and Bob</h1>

1. Required prior knowledge of the buffer size used, for all the parties.

In [57]:
buffer_size = 3200 # y of (y X y)
sha_key_str = 'Tuesday Evening'

2. QuantCrypt provides Kyber1024 key encapsulation mechanism.

In [58]:
from quantcrypt.kem import Kyber

kem = Kyber()

3. Alice generates key pair.

In [59]:
public_key, secret_key = kem.keygen()

4. Bob generates cipher and shared secret using Alice's public key.

In [60]:
cipher_text, shared_secret = kem.encaps(public_key)


5. Alice decrypts cipher she receives from Bob to extract shared secret.

In [61]:

shared_secret_copy = kem.decaps(secret_key, cipher_text)


6. Bob encrypts the message to be sent using the shared_secret.

In [62]:
from selective_encryption import encrypt
from utils import time_it

encrypt(file_path='data/files/landscape.png',
           sha_key_str=sha_key_str, 
           buffer=buffer_size,
           aes_key=shared_secret)

itr_count: 0
dwt2d: 0.050255250
dwt2d: 0.009504334
sha256_with_key: 0.000249541
sha512_with_key: 0.001113375
xor_with_sha_key: 0.001617750
xor_with_sha_key: 0.007035291
aes_encrypt: 0.002323208
_se_encrypt: 0.077191958
itr_count: 1
dwt2d: 0.031142625
dwt2d: 0.008920584
sha256_with_key: 0.000280917
sha512_with_key: 0.001091166
xor_with_sha_key: 0.001568041
xor_with_sha_key: 0.006816792
aes_encrypt: 0.002235667
_se_encrypt: 0.056424167
encrypt: 0.141295250


7. Alice decrypts the message using shared_secret_copy.

In [63]:
from selective_encryption import decrypt

decrypt(file_PF= 'data/files/enc/landscape.png_PF.npy',
           file_PPF_1= 'data/files/enc/landscape.png_PPF_1.npy',
           file_PPF_2= 'data/files/enc/landscape.png_PPF_2.npy',
           buffer=buffer_size,
           output_file_path='data/files/dec/landscape.png',
           sha_key_str=sha_key_str, 
           aes_key=shared_secret_copy)

aes_decrypt: 0.002177625
sha256_with_key: 0.000253125
xor_with_sha_key: 0.001995500
sha512_with_key: 0.001189750
xor_with_sha_key: 0.007507084
idwt2d: 0.008257250
idwt2d: 0.035539583
aes_decrypt: 0.002305166
sha256_with_key: 0.000262166
xor_with_sha_key: 0.001742750
sha512_with_key: 0.001100416
xor_with_sha_key: 0.007333208
idwt2d: 0.008148125
idwt2d: 0.034685833
decrypt: 2.541776750
